# Test LLM prompting for emotional evaluation given a set of paintings

In [3]:
#import libraries

from google import genai
from google.genai import types

import os
import re
import time
import random
import logging
from pathlib import Path

import pandas as pd
from tqdm import tqdm
from PIL import Image






ImportError: cannot import name 'genai' from 'google' (unknown location)

#### Get started 

In [148]:
#call client by passing api key I created 
# https://ai.google.dev/gemini-api/docs/api-key
client = genai.Client(api_key = api_key)


In [119]:
#example response
response = client.models.generate_content(
    model ="gemini-2.0-flash",contents = "Explain how AI works in a few words"
)

print(response.text)

AI uses algorithms to learn from data and make predictions or decisions.



##### Define Prompts

### Upload an Image File

In [88]:
#https://ai.google.dev/gemini-api/docs/image-understanding#upload-image
#Testing first with one image

with open('/Users/stellad/Desktop/Iigaya/CV/ImagesAttentionFrame/15_test/pillars-of-salt.jpg',"rb") as img_file:
    image_bytes = img_file.read()

response = client.models.generate_content(
    model = "gemini-2.5-flash",
    contents = [
        types.Part.from_bytes(
            data=image_bytes,
            mime_type = 'image/jpeg',
        ), 
        prompt_dict["Joy"]
                ]
)

print(response.text)

Joy: 8.5
Explanation: The emotion of joy is largely absent from this painting. The figures are depicted with stylized, often closed eyes and solemn or ambiguous facial expressions. There are no open smiles, laughter, or any indication of exuberant happiness typically associated with joy. Their postures are static, contemplative, and somewhat rigid, conveying a sense of introspection, reverence, or perhaps even sorrow, rather than delight or cheerfulness. The cool blue, green, and white tones used for the figures contribute to a serene, almost ethereal, and somewhat detached mood, which does not resonate with the vibrant energy of joy. While the dramatic red and orange streaks in the background add intensity, they evoke a powerful, possibly spiritual or foreboding atmosphere, rather than one of celebration. The only minute hint that could be vaguely interpreted as approaching a mild form of contentment, rather than outright joy, is the slight, almost imperceptible upward curve at the co

In [1]:
embed_result = client.models.embed_content(
    model = 'gemini-embedding-exp-03-07',
    contents = "The painting evokes a sense of mystery and intrigue. Its abstract forms and muted color palette create a somber and contemplative mood. The shapes are somewhat unsettling, leaving me with a feeling of unease and a desire to understand the hidden meaning behind the artwork."
)

NameError: name 'client' is not defined

/Users/stellad/Desktop/Iigaya/CV/ImagesAttentionFrame/20_block1

The painting evokes a sense of mystery and abstraction. The use of muted colors and the non-representational form create a dreamlike quality. There's a hint of mechanical or structural elements, but they are presented in a way that feels more organic and symbolic than literal. It invites interpretation and contemplation.

The painting evokes a sense of mystery and intrigue. Its abstract forms and muted color palette create a somber and contemplative mood. The shapes are somewhat unsettling, leaving me with a feeling of unease and a desire to understand the hidden meaning behind the artwork.

In [25]:
print(embed_result.model_dump())


{'embeddings': [{'values': [-0.007629887, -0.0019131314, 0.028974395, -0.08618827, 0.016005162, -0.01464117, 0.005047589, 0.0029754248, 0.011891935, 0.0075957817, -0.0036984456, 0.0023674183, -0.021635382, 0.031239023, 0.11759874, -0.01042253, -0.0041919434, 0.0106783835, 0.008832188, 0.007244372, -0.0027125243, -0.024487825, -0.006407545, -0.027267037, 0.0016433363, -0.007257276, 0.031546865, -0.015423704, 0.026210014, -0.02465299, 0.0008620344, 0.017538762, 0.0156998, 0.014690975, 0.04111737, -0.011956561, -0.0074187974, -0.029085696, -0.00023366016, 0.00962879, 0.0040264446, 0.03415735, -0.002885533, 0.017289797, 0.015240823, -0.019150784, 0.0122784795, 0.0063597313, -0.0045215893, 0.0123273665, 0.006644947, 0.014169254, -0.006970191, -0.15754595, 0.003605387, 0.00877179, -0.017750207, 0.01202259, 0.047812734, 0.009781829, -0.01744159, 0.03787559, 0.001976435, -0.011708151, -0.012938998, -0.00037750872, 0.020559972, -0.0032898372, -0.022242077, -0.029463938, 0.0082260845, -0.0226063

### Testing with prompts (numerical evaluation + text response explanation) for 20 images with Gemini
#### For each emotion, write one model to produce output. Plus one aesthetic rating model.

In [171]:
#Configuratoins
IMAGE_FOLDER = Path("/Users/stellad/Desktop/Iigaya/CV/ImagesAttentionFrame/10_test")
OUTPUT_CSV   = Path("emotion_model_output.csv")
MODEL_NAME   = "gemini-2.5-flash"  
MAX_RETRIES  = 3
BACKOFF_BASE = 1.0  # seconds
DELAY_BETWEEN_CALLS = 0.5  # throttle to avoid hitting rate limits


#manage situations where there is hidden files such as .DS_Store
IMAGE_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff"}

def is_image_file(path: Path):
    return path.is_file() and path.suffix.lower() in IMAGE_EXTS and not path.name.startswith(".")

In [172]:
#prompts
def build_emotion_prompt(emotion: str) -> str:
    return (
        f"You are an art expert describing your emotional response to a painting.\n"
        f"Evaluate **{emotion.lower()}** independently, without reference to any other feeling. "
        "Do not assume anything about other possible emotional reactions — focus only on this one emotion.\n\n"
        "Provide your response using the following structure:\n"
        "1. A **numeric score** between 0 and 100 (on a continuous scale — do not round to nearest 5 or 10 unless warranted)\n"
        "2. A **detailed explanation** supporting the reason behind the rating you provided. Please try to be as detailed as possible.\n\n"
        f"Use the format exactly:\n"
        f"{emotion}: [score]\n"
        "Explanation: ..."
    )

# Prompt dictionary for loop call models later on with all emotions and liking rating:
prompt_dict = {emotion: build_emotion_prompt(emotion) for emotion in ["Joy", "Sadness", "Fear", "Anger", "Disgust", "Surprise"]}
prompt_dict["Liking"] = (
    "You are an art expert evaluating how much you like a painting.\n"
    "Rate your **personal aesthetic preference** for the painting, based only on what is visually presented.\n"
    "Provide your response using the following structure:\n"
    "1. A **numeric score** between 0 and 100 (on a continuous scale — do not round unless appropriate)\n"
    "2. A **detailed explanation** supporting the reason behind the rating you provided. Please try to be as detailed as possible.\n\n"
    "Use the format exactly:\n"
    "Liking: [score]\n"
    "Explanation: ..."
)


In [173]:

# 2) Init GenAI client
client = genai.Client(api_key=api_key)

# 3) Regex to pull out the score and explanation
_SCORE_RE = re.compile(r"^[A-Za-z]+:\s*([\d.]+)", re.MULTILINE)
_EXPL_RE  = re.compile(r"Explanation:\s*(.*)", re.DOTALL)

def parse_response(text: str):
    m1 = _SCORE_RE.search(text)
    m2 = _EXPL_RE.search(text)
    if not m1 or not m2:
        return None, text.strip()
    return float(m1.group(1)), m2.group(1).strip()

# 4) Wrapped call with retry/backoff
def generate_with_retries(prompt: str, pil_img: Image.Image):
    last_exc = None
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            return client.models.generate_content(
                model=MODEL_NAME,
                contents=[prompt, pil_img],
            )
        except Exception as e:
            last_exc = e
            logging.warning(f"Attempt {attempt}/{MAX_RETRIES} failed: {e}")
            time.sleep(BACKOFF_BASE * (2 ** (attempt - 1)) + random.random())
    raise last_exc

# 5) Main loop
def main():
    records = []
    for img_path in sorted(IMAGE_FOLDER.iterdir()):
        if not is_image_file(img_path):
            continue

        row = {"image": img_path.name}
        pil_img = Image.open(img_path)

        for emotion, prompt in prompt_dict.items():
            try:
                resp = generate_with_retries(prompt, pil_img)
                text = resp.text
                score, expl = parse_response(text)
                if score is None:
                    row[f"{emotion.lower()}_rating"]      = ""
                    row[f"{emotion.lower()}_explanation"] = text.strip()
                else:
                    row[f"{emotion.lower()}_rating"]      = score
                    row[f"{emotion.lower()}_explanation"] = expl

            except Exception as e:
                logging.error(f"{img_path.name}–{emotion} ERROR: {e}")
                row[f"{emotion.lower()}_rating"]      = ""
                row[f"{emotion.lower()}_explanation"] = f"ERROR: {e}"

            time.sleep(DELAY_BETWEEN_CALLS)

        records.append(row)

    pd.DataFrame(records).to_csv(OUTPUT_CSV, index=False)
    print(f"✅ Done — results saved to {OUTPUT_CSV}")

In [174]:
if __name__ == "__main__":
    main()

KeyboardInterrupt: 

In [146]:
import inspect
print(inspect.signature(client.models.generate_content))

(*, model: str, contents: Union[list[Union[google.genai.types.Content, list[Union[google.genai.types.File, google.genai.types.Part, PIL.Image.Image, str]], google.genai.types.File, google.genai.types.Part, PIL.Image.Image, str]], google.genai.types.Content, list[Union[google.genai.types.File, google.genai.types.Part, PIL.Image.Image, str]], google.genai.types.File, google.genai.types.Part, PIL.Image.Image, str, list[Union[google.genai.types.Content, list[Union[google.genai.types.File, google.genai.types.Part, PIL.Image.Image, str]], google.genai.types.File, google.genai.types.Part, PIL.Image.Image, str, google.genai.types.ContentDict]], google.genai.types.ContentDict], config: Union[google.genai.types.GenerateContentConfig, google.genai.types.GenerateContentConfigDict, NoneType] = None) -> google.genai.types.GenerateContentResponse


In [ ]:
# Identify mime types of each image to be processed in the model

def get_mime_type(image_path: str) -> str:
    # Guess type, default to 'application/octet-stream' but we want image types
    mime_type, _ = mimetypes.guess_type(image_path)
    # Fallback for common cases:
    if mime_type is None:
        ext = os.path.splitext(image_path)[1].lower()
        if ext in ['.jpg', '.jpeg']:
            mime_type = 'image/jpeg'
        elif ext == '.png':
            mime_type = 'image/png'
        else:
            mime_type = 'application/octet-stream'
    return mime_type


In [109]:
# model
def emotion_model(client, image_path: str, prompt_text: str, emotion_label: str, max_output_tokens: int = 1024) -> dict:


    with open(image_path, "rb") as img_file:
        image_bytes = img_file.read()
    mime_type = get_mime_type(image_path)

    response = None

    for attempt in range(3):
        try:
            response = client.models.generate_content(
                model="gemini-2.5-flash",
                contents=[
                    types.Part.from_bytes(data=image_bytes, mime_type= mime_type),
                    prompt_text
                ],
                config=types.GenerateContentConfig(
                max_output_tokens=max_output_tokens)
                
            )
            #if response is not None but response.text is None, retry
            if response is not None and response.text:
                break
            else: 
                #log and retry
                print(f"Warning: Empty response.text on attempt {attempt+1} for {emotion_label}/{image_path}")
                response = None
                time.sleep(2**attempt)
        except Exception as e:
            print(f"Exception on attempt {attempt+1} for {emotion_label} / {image_path}: {e}")
            if attempt < 2:
                time.sleep(2 ** attempt)
            else:
                raise e
            
    if response is None:
        text = None
    else:
        text = response.text
        
    return{
        f"{emotion_label.lower()}_raw_response":text,
        **parse_response_single_emotion(text, emotion=emotion_label)
    }




In [110]:
# Ensure all model response outputs are formatted in the same way
def parse_response_single_emotion(response_text: str, emotion: str) -> dict:
    if not response_text:
        return {
            f"{emotion.lower()}_rating": None,
            f"{emotion.lower()}_explanation": "Empty response"
        }

    match = re.search(rf"{emotion}:\s*(\d+\.?\d*)\s*\nExplanation:\s*(.*)", response_text, re.DOTALL)
    if match:
        rating = float(match.group(1))
        explanation = match.group(2).strip()
    else:
        rating = None
        explanation = "Pattern not found"

    return {
        f"{emotion.lower()}_rating": rating,
        f"{emotion.lower()}_explanation": explanation
    }

In [111]:
def analyze_folder_to_csv(client, folder_path: str, output_csv_path: str, prompt_dict: dict):
    emotion_order = ["Joy", "Sadness", "Fear", "Anger", "Disgust", "Surprise", "Liking"]
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    # Initialize one row per image
    all_results = {filename: {"image_name": filename} for filename in image_files}

    for emotion in emotion_order:
        print(f"\n Running prompt: {emotion}")
        for filename in image_files:
            image_path = os.path.join(folder_path, filename)
            try:
                result = emotion_model(client, image_path, prompt_dict[emotion], emotion)
                all_results[filename].update(result)
            except Exception as e:
                print(f"Error processing {filename} for {emotion}: {e}")

    # Final fieldnames for CSV
    fieldnames = ['image_name']
    for emotion in emotion_order:
        fieldnames.extend([
            f"{emotion.lower()}_rating",
            f"{emotion.lower()}_explanation",
            f"{emotion.lower()}_raw_response"
        ])

    # Write to CSV
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(all_results.values())

    print(f"\n Saved {len(all_results)} results to: {output_csv_path}")



In [112]:
# Setup Gemini client
client = genai.Client(api_key=api_key)

# Define your emotion prompts
"""
prompt_dict = {
    "Joy": joy_prompt,
    "Sadness": sadness_prompt,
    "Fear": fear_prompt,
    "Anger": anger_prompt,
    "Disgust": disgust_prompt,
    "Surprise": surprise_prompt,
    "Liking": liking_prompt
}
"""


# Run batch processing with independent prompts per emotion
analyze_folder_to_csv(
    client=client,
    folder_path="/Users/stellad/Desktop/Iigaya/CV/ImagesAttentionFrame/15_test",
    output_csv_path="emotion_model_output.csv",
    prompt_dict=prompt_dict
)



 Running prompt: Joy
Exception on attempt 1 for Joy / /Users/stellad/Desktop/Iigaya/CV/ImagesAttentionFrame/15_test/pillars-of-salt.jpg: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}
Exception on attempt 2 for Joy / /Users/stellad/Desktop/Iigaya/CV/ImagesAttentionFrame/15_test/pillars-of-salt.jpg: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}


KeyboardInterrupt: 